In [99]:
import nltk, pickle, json, random
from nltk.stem import WordNetLemmatizer
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Biraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Biraj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [100]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!', ',']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [101]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

53 documents
14 classes ['about', 'care', 'care_response', 'figs', 'goodbye', 'greeting', 'joke', 'knock', 'laugh', 'nobel', 'notworking', 'stress', 'tank', 'thanks']
75 unique lemmatized words ["'m", "'s", 'a', 'again', 'alright', 'am', 'anyone', 'are', 'asking', 'awesome', 'boored', 'bye', 'call', 'can', 'chatting', 'day', 'do', 'exausted', 'fig', 'fine', 'for', 'from', 'funny', 'good', 'goodbye', 'haha', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'joke', 'kind', 'later', 'laugh', 'little', 'lol', 'make', 'me', 'mood', 'name', 'next', 'nice', 'no', 'nobel', 'nope', 'not', 'of', 'one', 'see', 'should', 'so', 'stressed', 'sure', 'tank', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'whats', 'who', 'work', 'yeah', 'you', 'your']
Training data created


<ipython-input-101-bc3c0de62dff>:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [102]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=20, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/500
3/3 [==============================] - 0s 1ms/step - loss: 2.6837 - accuracy: 0.0721
Epoch 2/500
3/3 [==============================] - 0s 2ms/step - loss: 2.6757 - accuracy: 0.0376
Epoch 3/500
3/3 [==============================] - 0s 1ms/step - loss: 2.6459 - accuracy: 0.0502
Epoch 4/500
3/3 [==============================] - 0s 1ms/step - loss: 2.6300 - accuracy: 0.0346
Epoch 5/500
3/3 [==============================] - 0s 1ms/step - loss: 2.5964 - accuracy: 0.1284
Epoch 6/500
3/3 [==============================] - 0s 1ms/step - loss: 2.5929 - accuracy: 0.0847
Epoch 7/500
3/3 [==============================] - 0s 998us/step - loss: 2.5838 - accuracy: 0.1255
Epoch 8/500
3/3 [==============================] - 0s 2ms/step - loss: 2.5407 - accuracy: 0.1787
Epoch 9/500
3/3 [==============================] - 0s 2ms/step - loss: 2.5079 - accuracy: 0.1756
Epoch 10/500
3/3 [==============================] - 0s 1000us/step - loss: 2.5353 - accuracy: 0.1348
Epoch 11/500
3/3 [======

3/3 [==============================] - 0s 2ms/step - loss: 1.0233 - accuracy: 0.7085
Epoch 85/500
3/3 [==============================] - 0s 999us/step - loss: 0.8262 - accuracy: 0.7711
Epoch 86/500
3/3 [==============================] - 0s 1000us/step - loss: 0.9171 - accuracy: 0.7649
Epoch 87/500
3/3 [==============================] - 0s 2ms/step - loss: 0.8244 - accuracy: 0.8588
Epoch 88/500
3/3 [==============================] - 0s 2ms/step - loss: 0.8927 - accuracy: 0.7524
Epoch 89/500
3/3 [==============================] - 0s 1ms/step - loss: 0.8449 - accuracy: 0.7775
Epoch 90/500
3/3 [==============================] - 0s 2ms/step - loss: 0.8109 - accuracy: 0.8182
Epoch 91/500
3/3 [==============================] - 0s 1000us/step - loss: 1.0464 - accuracy: 0.6584
Epoch 92/500
3/3 [==============================] - 0s 1ms/step - loss: 0.8727 - accuracy: 0.7430
Epoch 93/500
3/3 [==============================] - 0s 2ms/step - loss: 0.9489 - accuracy: 0.6553
Epoch 94/500
3/3 [=======

3/3 [==============================] - 0s 2ms/step - loss: 0.3405 - accuracy: 0.9592
Epoch 167/500
3/3 [==============================] - 0s 2ms/step - loss: 0.2533 - accuracy: 0.8966
Epoch 168/500
3/3 [==============================] - 0s 1ms/step - loss: 0.2757 - accuracy: 0.9404
Epoch 169/500
3/3 [==============================] - 0s 1ms/step - loss: 0.2455 - accuracy: 0.9185
Epoch 170/500
3/3 [==============================] - 0s 2ms/step - loss: 0.2126 - accuracy: 0.9718
Epoch 171/500
3/3 [==============================] - 0s 2ms/step - loss: 0.2659 - accuracy: 0.9436
Epoch 172/500
3/3 [==============================] - 0s 2ms/step - loss: 0.2578 - accuracy: 0.9624
Epoch 173/500
3/3 [==============================] - 0s 1000us/step - loss: 0.2997 - accuracy: 0.8527
Epoch 174/500
3/3 [==============================] - 0s 2ms/step - loss: 0.2972 - accuracy: 0.9153
Epoch 175/500
3/3 [==============================] - 0s 1000us/step - loss: 0.2175 - accuracy: 0.9843
Epoch 176/500
3/3 

3/3 [==============================] - 0s 2ms/step - loss: 0.1529 - accuracy: 0.9467
Epoch 248/500
3/3 [==============================] - 0s 1ms/step - loss: 0.1891 - accuracy: 0.9498
Epoch 249/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0889 - accuracy: 0.9843
Epoch 250/500
3/3 [==============================] - 0s 1ms/step - loss: 0.1137 - accuracy: 0.9843
Epoch 251/500
3/3 [==============================] - 0s 998us/step - loss: 0.1392 - accuracy: 0.9749
Epoch 252/500
3/3 [==============================] - 0s 1ms/step - loss: 0.1024 - accuracy: 1.0000
Epoch 253/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1059 - accuracy: 1.0000
Epoch 254/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1706 - accuracy: 0.9718
Epoch 255/500
3/3 [==============================] - 0s 1ms/step - loss: 0.1700 - accuracy: 0.9718
Epoch 256/500
3/3 [==============================] - 0s 1ms/step - loss: 0.1159 - accuracy: 0.9718
Epoch 257/500
3/3 [===

3/3 [==============================] - 0s 2ms/step - loss: 0.0308 - accuracy: 1.0000
Epoch 329/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0812 - accuracy: 0.9718
Epoch 330/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0672 - accuracy: 0.9906
Epoch 331/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 332/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1369 - accuracy: 0.9686
Epoch 333/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1894 - accuracy: 0.8903
Epoch 334/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0997 - accuracy: 0.9843
Epoch 335/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1128 - accuracy: 0.9404
Epoch 336/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0372 - accuracy: 0.9906
Epoch 337/500
3/3 [==============================] - 0s 1000us/step - loss: 0.0889 - accuracy: 1.0000
Epoch 338/500
3/3 [==

3/3 [==============================] - 0s 1ms/step - loss: 0.0490 - accuracy: 1.0000
Epoch 411/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0478 - accuracy: 1.0000
Epoch 412/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0412 - accuracy: 0.9843
Epoch 413/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1302 - accuracy: 0.9561
Epoch 414/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0285 - accuracy: 1.0000
Epoch 415/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0526 - accuracy: 1.0000
Epoch 416/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1153 - accuracy: 0.9718
Epoch 417/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0380 - accuracy: 1.0000
Epoch 418/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0781 - accuracy: 0.9718
Epoch 419/500
3/3 [==============================] - 0s 1ms/step - loss: 0.1320 - accuracy: 0.9718
Epoch 420/500
3/3 [=====

3/3 [==============================] - 0s 2ms/step - loss: 0.0700 - accuracy: 1.0000
Epoch 493/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0618 - accuracy: 1.0000
Epoch 494/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1154 - accuracy: 0.9467
Epoch 495/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0236 - accuracy: 1.0000
Epoch 496/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0637 - accuracy: 0.9718
Epoch 497/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0629 - accuracy: 0.9906
Epoch 498/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 499/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 500/500
3/3 [==============================] - 0s 1ms/step - loss: 0.0998 - accuracy: 0.9436
model created
